In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import youtube_credentials
import collections
import httplib2
import os
import sys

USER_NAME='22091989'

In [ ]:
credentials = youtube_credentials.get_credentials()
youtube = youtube_credentials.get_youtube()

In [ ]:
def get_channel_id_from_username(username):
    search_response = youtube.search().list(
        q=username,
        part="id",
        maxResults=1
    ).execute()

    response = search_response.get("items", [])
    assert len(response) == 1
    channel = response[0]
    channel_id = channel['id']['channelId']
    return channel_id

def get_playlists(channel_id, list_kwargs = dict()):
    search_response = youtube.playlists().list(
        part = 'id,snippet',
        channelId = channel_id,
        **list_kwargs
    ).execute()
    return search_response

def get_playlist_items(playlist_id, pageToken = None):
    # See: https://developers.google.com/youtube/v3/docs/playlistItems/inser t
    items = []
    while True:
        response = youtube.playlistItems().list(part = 'id,snippet,contentDetails', playlistId=p_id, maxResults=50, pageToken = pageToken).execute()
        items += response.get('items', [])
        if not 'nextPageToken' in response:
            break
        pageToken = response.get('nextPageToken')
    return items

def add_video_to_playlist(p_id, v_id, pos = 1):
    resource = {
        'snippet': {
            'playlistId': p_id,
            'position': pos,
            'resourceId': {
                'kind': 'youtube#video', 'videoId': v_id
            }
        }
    }
    return youtube.playlistItems().insert(
        body=resource,
        part='snippet'
    ).execute()

def copy_playlists(playlist_items, verbose = False, dry_run = False):
    for p_idx, (p_name, videos) in enumerate(playlist_items.items()):
        print('\tCreating playlist {:4}/{}: {}'.format(p_idx + 1, len(playlist_items), p_name))
        playlists_insert_response = youtube.playlists().insert(
          part="snippet,status",
          body=dict(
            snippet=dict(
              title=p_name,
              description=":)"
            ),
            status=dict(
              privacyStatus="public"
            )
          )
        ).execute()
        
        if not dry_run:
            playlist_id = playlists_insert_response.get('id', None)
            assert playlist_id
        else:
            playlist_id = 'DRY_RUN'

        for idx, v in enumerate(videos):
            try:
                pos = v['snippet']['position']
                v_id = v['contentDetails']['videoId']
                v_title = v['snippet']['title']
                
                if verbose:
                    print('\t\tAdding video {:4}/{}: {}'.format(idx + 1, len(videos), v_title))
                
                if not dry_run:
                    response = add_video_to_playlist(playlist_id, v_id, pos)
            except Exception as e:
                print('\t\tError adding video: {}'.format(e))

def copy_videos_from_user_to_own_channel(username, dry_run = False):
    print('Retrieving user id for: {}'.format(username))
    channel_id = get_channel_id_from_username(username)
    print('Retrieving playlists')
    playlists = get_playlists(channel_id, list_kwargs=dict(maxResults = 50)).get('items', [])
    playlist_items = {playlist['snippet']['title']: get_playlist_items(playlist['id']) for playlist in playlists}
    print('Starting to copy playlists')
    copy_playlists(playlist_items, verbose = False, dry_run = dry_run)
    print('Finished')

In [ ]:
copy_videos_from_user_to_own_channel(USER_NAME, dry_run = True)